# 2. WanDB 적용해보기

여기에서는 baseline에 WanDB를 적용하여 Experiment tracking 기능을 활용해보도록 하겠습니다.

즉, 다시 말하면 실험에 대한 정보를 WanDB 페이지상에 전송하는 것입니다.

## 1. 설치하기 및 로그인하기

In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

WanDB의 경우 여러가지 방식으로 동작시킬 수 있습니다.

Commend line을 통한 방법과 python 코드를 통하는 방법입니다. 하지만 결과 자체는 동일합니다.

[Commend line](https://docs.wandb.ai/ref/cli/wandb-login)

[Github](https://github.com/wandb/wandb/blob/latest/wandb/sdk/wandb_login.py#L43-L77)

로그인 역시 마찬가지로 로그인 이후에는 해당 터미널/세션이 끝나기 전까진 별도의 로그인 작업 없이 진행할 수 있습니다.

### **wandb login [option] [key]**

In [3]:
# 커맨드 라인을 통한 로그인 방식

!wandb login [token key] # 이와 같이 미리 토큰 값을 입력할 수 있습니다.자세한 옵션은 아래 링크 참조

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
# 코드를 이용한 방식 이 방법을 추천

# wandb.login()

## 2. Baseline 구현(데이터 준비 ~ 전처리)

데이터 분석 파트는 생략하도록 하겠습니다.

In [2]:
# Data load

cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
# Validation Set

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25)

In [4]:
CLASS_NAMES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "hores", "ship", "truck"]

In [5]:
# Normalization

X_train, X_val, X_test = X_train / 255., X_val / 255., X_test / 255.

## 3. wandb.init() 알아보기

[commend line](https://docs.wandb.ai/ref/cli/wandb-init)

[python code](https://docs.wandb.ai/ref/python/init)

[github](https://github.com/wandb/wandb/blob/latest/wandb/sdk/wandb_init.py#L722-L990)

### config에 대해서

config에는 모델에 대한 가능한 많은 정보를 기입하기를 추천합니다.

config의 경우 yaml 파일 형식으로 많이 활용합니다.

[python code](https://docs.wandb.ai/ref/python/config)

In [6]:
run = wandb.init(project = 'keras-example',
                 entity="suncreation",
                 config = {
                     "n_channel_1" : 32,
                     "n_channel_2" : 64,
                     "n_dense" : 1024,
                     "learning_rate" : 0.005,
                     "epochs" : 5,
                     "batch_size" : 32,
                     "optimizer" : "adam",
                     "loss" : "sparse_categorical_crossentropy",
                     "metrics" : ["accuracy"]
                 })

config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: suncreation. Use `wandb login --relogin` to force relogin


## 4. wandb.config 적용하기

In [7]:
# model 설계

tf.keras.backend.clear_session()

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(config.n_channel_1, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(config.n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(config.n_dense, activation='relu'))
model.add(keras.layers.Dense(len(CLASS_NAMES), activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

Model에 추가된 Layer 개수:  7


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2360320   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [9]:
if config.optimizer == "adam":
    optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
else:
    optimizer = keras.optimizers.SGD()

model.compile(optimizer = optimizer,
             loss = config.loss,
             metrics = config.metrics)

## 5. WandbCallback

[keras wandbcallback](https://docs.wandb.ai/ref/python/integrations/keras/wandbcallback)

[github](https://github.com/wandb/wandb/blob/latest/wandb/integration/keras/keras.py#L272-L969)

주요 prams
*   monitor
*   log_weights / gradients

Visualization 시 parms
*   validation_data
*   input_type(data_type)
*   labels
*   output_type



In [10]:
# 아래의 val_image는 WandbCallback에 사용하기 위한 용도입니다.

val_images, val_labels = X_test[:32], y_test[:32]

model.fit(X_train, y_train,
          epochs = config.epochs,
          batch_size = config.batch_size,
          validation_data = (X_val, y_val),
          callbacks = [WandbCallback(input_type = 'images',
                                     training_data = (val_images, val_labels),
                                     labels = CLASS_NAMES,
                                     log_weights = True,
                                     log_gradients = True)],)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/5
1172/1172 [==============================] - 9s 5ms/step - loss: 1.6408 - accuracy: 0.4042 - val_loss: 1.4818 - val_accuracy: 0.4697
INFO:tensorflow:Assets written to: /aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best)... Done. 0.2s


Epoch 2/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.3721 - accuracy: 0.5087 - val_loss: 1.5615 - val_accuracy: 0.4650
Epoch 3/5
1172/1172 [==============================] - 6s 5ms/step - loss: 1.2691 - accuracy: 0.5511 - val_loss: 1.3003 - val_accuracy: 0.5424
INFO:tensorflow:Assets written to: /aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best)... Done. 0.2s


Epoch 4/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.1728 - accuracy: 0.5864 - val_loss: 1.2995 - val_accuracy: 0.5408
INFO:tensorflow:Assets written to: /aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best)... Done. 0.2s


Epoch 5/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.0864 - accuracy: 0.6191 - val_loss: 1.2983 - val_accuracy: 0.5591
INFO:tensorflow:Assets written to: /aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/wandb/run-20220810_085019-1sxvxgse/files/model-best)... Done. 0.2s


## 6. wandb.log

[python code](https://docs.wandb.ai/ref/python/log)

[github](https://github.com/wandb/wandb/blob/latest/wandb/sdk/wandb_run.py#L1125-L1351)

In [11]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
           "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})

313/313 - 1s - loss: 1.2931 - accuracy: 0.5544
test_loss: 1.2930829524993896 
test_accuracy: 0.5544000267982483


In [12]:
wandb.finish()

Test Accuracy Rate:,▁
Test Error Rate:,▁
accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▁▇▇█
val_loss,▆█▁▁▁
GFLOPs,0.0063
Test Accuracy Rate:,55.44
Test Error Rate:,44.56
accuracy,0.61912
